In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from ytmusicapi import YTMusic
import logging
import os

# Assumes you've created environmental variables from https://developer.spotify.com/dashboard/
CID = os.environ.get('SPOTIFY_CLIENT_ID')
CS = os.environ.get('SPOTIFY_CLIENT_SECRET')

auth_manager = SpotifyClientCredentials(client_id=CID,client_secret=CS)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Assumes you followed: https://ytmusicapi.readthedocs.io/en/latest/setup.html
yt = YTMusic('headers_auth.json')

logging.basicConfig(filename='songs.log', filemode='w', format='%(levelname)s - %(message)s', level=logging.INFO) 

In [6]:
SPOTIFY_PLAYLIST_ID = 'SPID'

# If new YouTube playlist
YOUTUBE_PLAYLIST_NAME = 'NAME'
YOUTUBE_PLAYLIST_DESCRIPTION = 'DESC'
YOUTUBE_PLAYLIST_ID = yt.create_playlist(YOUTUBE_PLAYLIST_NAME,YOUTUBE_PLAYLIST_DESCRIPTION)

# Switch commenting if you are adding to an already existing YouTube playlist
# YOUTUBE_PLAYLIST_ID = "YPID"

In [7]:
# Check the number of time's we'll need to loop based on the # of songs in the playlist
iterations = range(int(sp.playlist_items(SPOTIFY_PLAYLIST_ID,fields='total')['total']/100) + 1)

playlist_tracks = []

# Max requests at one time is 100, so we increment by that
offset = 0
for i in iterations:
    playlist_tracks.extend(sp.playlist_items(SPOTIFY_PLAYLIST_ID, fields='items.track.name, items.track.artists', offset=offset)['items'])
    offset += 100

In [8]:
# Parse the song dictionary to create a search string using the song name and artists. Example: Return to Oz by Monolink, ARTBAT
def get_song_search_string(song_dict: dict) -> str:
    song_name = song_dict['track']['name'].split(' (')[0]
    song_artists = ', '.join([artist['name'] for artist in song_dict['track']['artists']])
    return(f"{song_name} by {song_artists}")

In [9]:
# Try to find the videoId returned by YouTube
def parse_id(songs: list) -> str:
    for song in songs:
        id = song.get('videoId')
        if song.get('videoId'):
            return id
        return None

In [10]:
# TODO: Add the option to batch add songs to help reduce rate limit threshold. Seems like 5 is the max number of id's to send at once
def batch_add(song_list: list, batch_number: int=5) -> None:
    if batch_number is not batch_add.__defaults__[0]:
        batch_number = batch_number
    return yt.add_playlist_items(YOUTUBE_PLAYLIST_ID, videoIds=song_list)

In [11]:
for track in playlist_tracks:
    song_search_string = get_song_search_string(track)
    song_results = yt.search(query=song_search_string, filter='songs')
    video_id = parse_id(song_results)
    if video_id:
        resp = yt.add_playlist_items(YOUTUBE_PLAYLIST_ID, videoIds=[video_id])
        status = resp['status']
        if status == 'STATUS_SUCCEEDED':
            logging.info(f'Succesfully added: {song_search_string}')
        else:
            error_text = resp['actions'][0]['addToToastAction']['item']['notificationActionRenderer']['responseText']['runs'][0]['text']
            logging.error(f'Could not add: {song_search_string} due to: {error_text}')
        continue
    logging.info(f'Could not find videoId for {song_search_string} in {len(song_results)} results.')
logging.info(f'Finished adding {len(playlist_tracks)} to {YOUTUBE_PLAYLIST_NAME} - {YOUTUBE_PLAYLIST_ID}')